In [156]:
from utils import *
import os
import pandas as pd
import numpy as np

In [2]:
data_dir = "data"
repo = "https://github.com/JeffSackmann/tennis_atp"
master = "JeffSackmann/tennis_atp/blob/master"
prefix = "https://raw.githubusercontent.com"

In [18]:
files = parser_csv(repo)
files_links = [os.path.join(master,i) for i in files]

In [139]:
match_files=[i for i in files if 'matches' in i]
ranking_files=[i for i in files if 'rankings' in i]

In [24]:
match_links=[i for i in files_links if 'matches' in i]
ranking_links=[i for i in files_links if 'rankings' in i]
player_link=[i for i in files_links if i not in match_links+ranking_links]

In [40]:
def fetch_data(links,prefix=None):
    l = [os.path.join(prefix,
                     re.sub(r'^/', '',file, count=1)) for file in links]
    urls = [re.sub(r'.blob', '',file, count=1) for file in l]
    dfs = [pd.read_csv(file) for file in urls]
    return pd.concat(dfs, ignore_index=True)

### Player Data

In [307]:
df_player=fetch_data(player_link, "https://raw.githubusercontent.com")
df_player=df_player.loc[(df_player.name_first != "") | (df_player.name_last != ""),]
first = df_player.name_first.isna() | (df_player.name_first == "")
second = df_player.name_last.isna() | (df_player.name_last == "")
df_player=df_player.loc[~(first & second),:]

In [292]:
first = df_player.name_first.isna() | (df_player.name_first == "")
second = df_player.name_last.isna() | (df_player.name_last == "")
hand = df_player.hand.isna() | (df_player.hand == "")
df_player['name_invalid']=first | second
df_player.loc[hand,'hand'] = 'U'
df_player['name_full'] = df_player.name_first+' '+df_player.name_last

In [293]:
df_player['year_invalid']=False
df_player['month_invalid']=False
df_player['day_invalid']=False

In [301]:
df_player.loc[~df_player.dob.isna(),'dob']=df_player.dob.loc[~df_player.dob.isna()].astype(int).astype(str)

In [281]:
def date_cleaner(x):
    if pd.isna(x['dob']):
        x['year_invalid']=True
        x['month_invalid']=True
        x['day_invalid']=True
        return x
    if not bool(re.search(r'^\d{8}$|\d{4}/\d{2}/\d{2}', x['dob'])):
        x['year_invalid']=True
        x['month_invalid']=True
        x['day_invalid']=True
        return x
    
    
    
    
    try:
        x['dob']=datetime.strptime(x['dob'], '%Y%m%d').strftime('%Y/%m/%d')
        return x
        
    except ValueError:
        if bool(re.search(r'^[3-9]', x['dob'])):
            x['dob']=re.sub(r'^[3-9]',r'1', x['dob'])
            x['year_invalid']=True
            
        if bool(re.search(r'^2[1-9]', x['dob'])):
            x['year_invalid']=True
        
        if bool(re.search(r'^20[3-9]', x['dob'])):
            x['year_invalid']=True
    
        if bool(re.search(r'\d{4}0{2}\d{2}', x['dob'])):
            x['dob']=re.sub(r'(\d{4})0{2}(\d{2})',r'\g<1>01\2', x['dob'])
            x['month_invalid']=True

        if bool(re.search(r'\d{4}[2-9]\d{3}', x['dob'])):
            x['dob']=re.sub(r'(\d{4})[2-9](\d{3})',r'\g<1>0\2', x['dob'])
            x['month_invalid']=True
        if bool(re.search(r'\d{4}1[3-9]\d{2}', x['dob'])):
            x['dob']=re.sub(r'(\d{4})1([3-9])(\d{2})',r'\g<1>0\2\3', x['dob'])
            x['month_invalid']=True

        if bool(re.search(r'\d{6}0{2}', x['dob'])):
            x['dob']=re.sub(r'(\d{6})00',r'\g<1>01', x['dob'])
            x['day_invalid']=True

        if bool(re.search(r'\d{6}[4-9]\d', x['dob'])):
            x['dob']=re.sub(r'(\d{6}[4-9](\d)',r'\g<1>0\2', x['dob'])
            x['day_invalid']=True

        if bool(re.search(r'\d{6}3[2-9]', x['dob'])):
            x['dob']=re.sub('(\d{6})3([2-9])',r'\g<1>0\2', x['dob'])
            x['day_invalid']=True
        
        try:
            x['dob']=datetime.strptime(x['dob'], '%Y%m%d').strftime('%Y/%m/%d')
            return x
        except ValueError:
            return x
    

df_player.to_csv('atp_players.csv', index=False)

In [306]:
df_player.loc[df_player.month_invalid & ~df_player.year_invalid,]

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,name_invalid,name_full,year_invalid,month_invalid,day_invalid
11844,111860,Ronald Victor,Thomas,U,1889/01/01,AUS,NaN,NaN,False,Ronald Victor Thomas,False,True,True
11862,111878,Rice Thomas Hopkins,Gemmell,U,1896/01/01,AUS,NaN,NaN,False,Rice Thomas Hopkins Gemmell,False,True,True
11883,111899,Bruce,Dive,U,1894/01/01,AUS,NaN,NaN,False,Bruce Dive,False,True,True
11895,111911,Andrew R,Huthnance,U,1899/01/01,AUS,NaN,NaN,False,Andrew R Huthnance,False,True,True
11899,111915,Jack,Cummings,U,1901/01/01,AUS,NaN,NaN,False,Jack Cummings,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15670,115686,Frank,Sulloway,U,1883/01/01,USA,NaN,NaN,False,Frank Sulloway,False,True,True
15887,115903,Edwin P,Fischer,U,1873/01/01,USA,NaN,NaN,False,Edwin P Fischer,False,True,True
16065,116081,John A,Ryerson,U,1862/01/01,USA,NaN,NaN,False,John A Ryerson,False,True,True
16481,116497,Calhoun,Cragin,U,1872/01/01,USA,NaN,NaN,False,Calhoun Cragin,False,True,True


In [303]:
df_player=df_player.apply(lambda x: date_cleaner(x),axis=1)


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [309]:
df_player.to_csv('atp_players.csv', index=False)

### Rankings Data